In [ ]:
from google.colab import files
files.upload()

Saving car-sales-extended-missing-data.csv to car-sales-extended-missing-data.csv


{'car-sales-extended-missing-data.csv': b'Make,Colour,Odometer (KM),Doors,Price\nHonda,White,35431.0,4.0,15323.0\nBMW,Blue,192714.0,5.0,19943.0\nHonda,White,84714.0,4.0,28343.0\nToyota,White,154365.0,4.0,13434.0\nNissan,Blue,181577.0,3.0,14043.0\nHonda,Red,42652.0,4.0,23883.0\nToyota,Blue,163453.0,4.0,8473.0\nHonda,White,,4.0,20306.0\n,White,130538.0,4.0,9374.0\nHonda,Blue,51029.0,4.0,26683.0\nNissan,White,167421.0,4.0,16259.0\nNissan,Green,17119.0,4.0,6160.0\nNissan,White,102303.0,4.0,16909.0\n,White,134181.0,4.0,11121.0\nHonda,Blue,199833.0,4.0,18946.0\nToyota,Blue,205592.0,4.0,16290.0\nToyota,Red,96742.0,4.0,34465.0\nBMW,White,194189.0,5.0,17177.0\nNissan,White,67991.0,3.0,9109.0\nNissan,Blue,215820.0,4.0,6010.0\nToyota,,124844.0,4.0,24130.0\nHonda,,30615.0,4.0,29653.0\nToyota,White,148744.0,4.0,22489.0\nHonda,Green,130075.0,4.0,21242.0\nHonda,Blue,172718.0,4.0,14274.0\nHonda,Blue,125819.0,4.0,15686.0\nHonda,White,180390.0,4.0,13344.0\nHonda,Green,82783.0,4.0,10984.0\nHonda,White,56

In [ ]:
import pandas as pd
car_sales_missing=pd.read_csv("/content/car-sales-extended-missing-data.csv")
car_sales_missing.head(10)

In [ ]:
car_sales_missing.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [ ]:
car_sales_missing.dropna(subset=["Price"],inplace=True)

In [ ]:
x = car_sales_missing.drop("Price", axis=1)
y = car_sales_missing["Price"]

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
num_imputer = SimpleImputer(strategy="mean")

cat_features = ["Make", "Colour"]
door_feature= ["Doors"]
num_features=["Odometer (KM)"]

imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, num_features)
])

filled_x=imputer.fit_transform(x)
filled_x

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [ ]:
car_sales_filled = pd.DataFrame(filled_x,
                                columns=["Make","Colour", "Doors", "Odometer (KM)"])
car_sales_filled.head()

In [ ]:
car_sales_filled.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features=["Make", "Colour", "Doors"]
one_hot=OneHotEncoder()
transformer=ColumnTransformer([("one_hot",
                                 one_hot,
                                 categorical_features)],
                                 remainder="passthrough")

transformed_x=transformer.fit_transform(car_sales_filled)
transformed_x

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [ ]:
import numpy as np
np.random.seed(42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(transformed_x,y,test_size=0.2)

model=RandomForestRegressor()
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.21990196728583944